# IDS projekt

## 1. Andmetöötlus


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive/')


print("moving to 'My Drive'")
os.chdir("drive/My Drive/")

Mounted at /content/drive/
moving to 'My Drive'


### 1.1 Exploring raw data

In [3]:
#
project_dir = "ids_projekt"
files = os.listdir(project_dir)
files.sort()

general_data_dir = files[0]
files = files[1:]


files[:5]


['tasutud_maksud_2020_i_kvartal_eng.csv',
 'tasutud_maksud_2020_ii_kvartal_eng.csv',
 'tasutud_maksud_2020_iii_kvartal_eng.csv',
 'tasutud_maksud_2020_iv_kvartal_eng.csv',
 'tasutud_maksud_2021_i_kvartal_eng.csv']

In [4]:
# Looking into I quarter 2020
i_quarter_2020 = files[0]

import chardet

# Read a small chunk of the file to analyze its encoding
with open(f"{project_dir}/{i_quarter_2020}", 'rb') as file:
    raw_data = file.read(10000)  # Read the first 10,000 bytes
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    print("Detected encoding:", encoding)

Detected encoding: ISO-8859-1


In [5]:
# Reading data
import pandas as pd

file_params = {
    'encoding': encoding,
    'sep': ';'
}

df_i_2020 = pd.read_csv(f"{project_dir}/{i_quarter_2020}", **file_params)
df_i_2020.head()

,Registry code,Name,Type,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
0,10000018,AMSERV AUTO AS,Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"635 586,56","669 944,03",NaN,208.0
1,10000024,"EESTI RAAMAT, OÜ",Company,yes,INFORMATION AND COMMUNICATION,Harju ( Tallinn ),"22 847,67","14 375,33","129 993,24",10.0
2,10000062,ALDO KOPPEL,Self-employed person,yes,"AGRICULTURE, FORESTRY AND FISHING",Ida-Viru ( Lüganuse vald ),NaN,NaN,"185,00",NaN
3,10000127,"ARAVETE APTEEK, TÜ",Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Järva ( Järva vald ),"8 717,01","4 503,35","116 415,20",2.0
4,10000165,"KIVIÕLI KAUBAHOOV, AS",Company,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Ida-Viru ( Lüganuse vald ),"90 795,72","57 953,73","842 647,07",45.0


In [6]:
df_i_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142890 entries, 0 to 142889
Data columns (total 10 columns):
 #   Column                                                                                                                                       Non-Null Count   Dtype  
---  ------                                                                                                                                       --------------   -----  
 0   Registry code                                                                                                                                142890 non-null  object 
 1   Name                                                                                                                                         142890 non-null  object 
 2   Type                                                                                                                                         142890 non-null  object 
 3   Registered in national VAT re

In [15]:
# Different entity types
df_i_2020['Type'].value_counts()

,count
Type,
Company,119920
Non-profit association,12375
Self-employed person,4148
Non-resident,4047
Government or state authority,1925
Foundation,473
International organisation located in Estonia,2


In [16]:
# Example of different types of legal entities in Estonia
df_i_2020.groupby('Type').first()

,Registry code,Name,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
Type,,,,,,,,,
Company,10000018,AMSERV AUTO AS,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"635 586,56","669 944,03","129 993,24",208.0
Foundation,90000050,ESTONIAN-REVELIA ACADEMIC FUND'I SIHTASUTUS EE...,no,OTHER SERVICE ACTIVITIES,Harju ( Tallinn ),"198,75","198,75","23 548,70",1.0
Government or state authority,70000042,PÕLLUMAJANDUSUURINGUTE KESKUS,yes,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",Harju ( Saku vald ),"1 073,40","0,00","843,12",167.0
International organisation located in Estonia,98000012,BALTI KAITSEKOLLEDÞ,no,EDUCATION,Tartu ( Tartu linn ),"187 846,92","197 796,97",None,38.0
Non-profit association,80000037,EESTI SUURPEREDE ABISTAMISE SELTS (LIKVIDEERIM...,no,OTHER SERVICE ACTIVITIES,Harju ( Tallinn ),"307,41","321,42","42 395,00",1.0
Non-resident,60000066,LEDVANCE OY,yes,WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VE...,Harju ( Tallinn ),"186 973,21","13 640,58","1 588 190,12",2.0
Self-employed person,10000062,ALDO KOPPEL,yes,"AGRICULTURE, FORESTRY AND FISHING",Ida-Viru ( Lüganuse vald ),"1 308,94","691,44","185,00",1.0


In [21]:
# Some outliers that don't really fit our purposes
df_i_2020[df_i_2020['Type'] == 'International organisation located in Estonia']

,Registry code,Name,Type,Registered in national VAT register,"Field of activity in EMTAK (Estonian Classification of Economic Activities), which is marked with an alphabetic code in the system of EMTAK",County,State taxes,Labour taxes and payments,Turnover,Number of employees
142766,98000012,BALTI KAITSEKOLLEDÞ,International organisation located in Estonia,no,EDUCATION,Tartu ( Tartu linn ),"187 846,92","197 796,97",NaN,38.0
142767,98000026,KOOPERATIIVNE KÜBERKAITSE KOMPETENTSIKESKUS,International organisation located in Estonia,no,NaN,Harju ( Tallinn ),"2 012,78","2 182,18",NaN,NaN


In [23]:
# Employee count data overview for each type of legal entity
df_i_2020.groupby('Type').agg(
    min_employees=('Number of employees', 'min'),
    max_employees=('Number of employees', 'max'),
    #range_employees=('Number of employees', lambda x: x.max() - x.min()),
    null_count=('Number of employees', lambda x: x.isnull().sum())
)

,min_employees,max_employees,null_count
Type,,,
Company,1.0,3738.0,40042
Foundation,1.0,4537.0,53
Government or state authority,1.0,4527.0,12
International organisation located in Estonia,38.0,38.0,1
Non-profit association,1.0,291.0,2680
Non-resident,1.0,331.0,3486
Self-employed person,1.0,18.0,2745


In [50]:
# Self-employed person legal entity type employee counts
df_i_2020[df_i_2020['Type'] == 'Self-employed person'].value_counts('Number of employees')

,count
Number of employees,
1.0,346
2.0,80
3.0,30
4.0,19
6.0,15
5.0,13
8.0,7
7.0,6
10.0,5


In [17]:
filter_values = ['Company', 'Non-profit association', 'Government or state authority']

# Filter the DataFrame
filtered_df = df_i_2020[df_i_2020['Type'].isin(filter_values)]
filtered_df = filtered_df.dropna(subset=['Turnover'])
filtered_df.describe()


,Number of employees
count,58398.000000
mean,8.755060
std,51.441966
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,4527.000000


In [39]:
general_data_dir = f"{project_dir}/{general_data_dir}"
[general_data_file] = os.listdir(general_data_dir)


df_general_data = pd.read_parquet(f"{general_data_dir}/{general_data_file}")
df_general_data.head()

,ariregistri_kood,nimi,yldandmed
0,16372442,000 Holdings OÜ,"{'aadressid': [{'aadress_ads__adob_id': None, ..."
1,12754230,001 group OÜ,{'aadressid': [{'aadress_ads__adob_id': '65438...
2,12652512,001 Kinnisvara OÜ,{'aadressid': [{'aadress_ads__adob_id': '65438...
3,16752073,007 Agent & Partners OÜ,{'aadressid': [{'aadress_ads__adob_id': '10262...
4,11694365,007 Autohaus osaühing,{'aadressid': [{'aadress_ads__adob_id': '86699...


In [49]:
import json
import numpy as np


example_data = df_general_data['yldandmed'][0]
def yldandmed_serializer(obj):
    if isinstance(obj, np.ndarray):  # Convert ndarray to list
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# Serialize with custom handler
pretty_json = json.dumps(example_data, indent=4, default=yldandmed_serializer)

print(pretty_json)


{
    "aadressid": [
        {
            "aadress_ads__adob_id": null,
            "aadress_ads__adr_id": null,
            "aadress_ads__ads_normaliseeritud_taisaadress": null,
            "aadress_ads__ads_normaliseeritud_taisaadress_tapsustus": null,
            "aadress_ads__ads_oid": null,
            "aadress_ads__koodaadress": null,
            "aadress_ads__tyyp": null,
            "algus_kpv": "01.02.2023",
            "ehak": null,
            "ehak_nimetus": null,
            "kaardi_nr": 1,
            "kaardi_piirkond": 5,
            "kaardi_tyyp": "R",
            "kande_nr": 2,
            "kirje_id": 10461173,
            "lopp_kpv": null,
            "postiindeks": "2010",
            "riik": "AUS",
            "riik_tekstina": "Austraalia",
            "tanav_maja_korter": "313A/133 GOULBURN STREET, Surry Hills, NSW"
        }
    ],
    "arinimed": [
        {
            "algus_kpv": "23.11.2021",
            "kaardi_nr": 1,
            "kaardi_piirkond": 5,
    

### 1.2 Initial thoughts

1. Moving forward should bring in the [open company data](https://avaandmed.ariregister.rik.ee/en/downloading-open-data)
2. Registry code can be used to identify companies even if they have changed name, maybe namechange is an indicator if company is gonna do well or not :D ?
3. Should we exclude estonian `FIE` (Self-Employed Person) since that's not what we are looking for?
4. Should have functions that give us parsed data, eg. We want to test our predictions on 3rd quarter 2024 so we read in data till the prev quarter?
5. Should we normalize and adjust things for inflation?
6. What more data can we include, eg. last company name change, median number of employees or sliding window employee change?
7. What else intresting can we find, maybe court cases, maybe business licences eg. alcohol sales?


### 1.3 TODO:
- [ ] Bring in other data
- [ ] Make functions that read into dataframes
- [ ] Start figuring out all the data we can gather here
- [ ] Put down formulae
- [ ] Normalize for inflation, account for minimum wage in that calendar year?
- [ ] In the end we want the model to act like a black box right, someone enters a company name or registry code, we collect info and feed into model for prediction

In [56]:
# Functions


# Estonian salary calculations helpers and notes
gross_salary = 3000
print(f"Calculating salary breakdown for gross_salary of {gross_salary}")
social_tax = gross_salary * 0.33
employer_unemployment_insurance_premium = gross_salary * 0.008

employee_unemployment_insurance_premium = gross_salary * 0.016
# funded_pension could be also 0 if people have opted out
funded_pension = gross_salary * 0.02
income_tax = (gross_salary - employee_unemployment_insurance_premium - funded_pension) * 0.2

salary_fund = gross_salary + social_tax + employer_unemployment_insurance_premium

take_home_pay = gross_salary - income_tax - employee_unemployment_insurance_premium - funded_pension
print(f"Salary fund is {salary_fund}")
print(f"Social tax is {social_tax}")
print(f"Employer unemployment insurance premium is {employer_unemployment_insurance_premium}")
print(f"Gross salary is {gross_salary}")
print(f"Funded pension is {funded_pension}")
print(f"Employee unemployment insurance premium is {employee_unemployment_insurance_premium}")
print(f"Income tax is {income_tax}")
print(f"Take home pay is {take_home_pay}")
# Takes as input the Labour taxes and payments and calculates salary fund
def maksud_tööandja_kuluks(tööjõumaksud):
    return 4014

assert maksud_tööandja_kuluks(1640.4) == 4014

Calculating salary breakdown for gross_salary of 3000
Salary fund is 4014.0
Social tax is 990.0
Employer unemployment insurance premium is 24.0
Gross salary is 3000
Funded pension is 60.0
Employee unemployment insurance premium is 48.0
Income tax is 578.4
Take home pay is 2313.6
